In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

df = pd.read_csv("../Data/2.sample_dataset.csv")
df = df.dropna(subset=['words'])

# Divisez le DataFrame en ensembles d'entraînement et de test
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Tokenisation des textes
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['words'])

# Convertir les textes en séquences d'entiers
train_sequences = tokenizer.texts_to_sequences(train_data['words'])
test_sequences = tokenizer.texts_to_sequences(test_data['words'])

# Rembourrage des séquences pour avoir des tailles égales
max_length = max(max(len(seq) for seq in train_sequences), max(len(seq) for seq in test_sequences))
train_data_pad = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_data_pad = pad_sequences(test_sequences, maxlen=max_length, padding='post')


2024-01-25 09:19:51.507820: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# Paramètres du modèle
embedding_dim = 100
vocab_size = len(tokenizer.word_index) + 1

# Création du modèle
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(1, activation='sigmoid'))

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [4]:
# Définir les labels d'entraînement et de test

train_labels = train_data['target'].values
test_labels = test_data['target'].values

# Entraîner le modèle
model.fit(train_data_pad, train_labels, epochs=5, validation_data=(test_data_pad, test_labels))

Epoch 1/5
24888/24888 [==============================] - 6370s 256ms/step - loss: 0.4641 - accuracy: 0.7793 - val_loss: 0.4473 - val_accuracy: 0.7906
Epoch 2/5
 3467/24888 [===>..........................] - ETA: 1:30:20 - loss: 0.3907 - accuracy: 0.8257

KeyboardInterrupt: 